In [ ]:

!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets

In [ ]:
import pandas as pd
import json
import numpy as np
import torch
import gzip
import shutil
from unsloth import FastLanguageModel, is_bfloat16_supported
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments, TextStreamer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!



<details open>
  <summary>Explicação das libs</summary>


```python
from unsloth import FastLanguageModel, is_bfloat16_supported
```


Unsloth é uma biblioteca otimizada para treinar modelos de linguagem grandes (LLMs) de forma mais rápida e eficiente. A função
- FastLanguageModel é usada para carregar modelos de linguagem pré-treinados com otimizações que tornam o fine-tuning mais rápido. Já o método is_bfloat16_supported verifica se o hardware suporta o formato de ponto flutuante bfloat16, útil para acelerar o treino em GPUs modernas.
- FastLanguageModel: Carrega e configura o modelo para treinamento com adaptações como LoRA (Low-Rank Adaptation) e quantização eficiente.
is_bfloat16_supported: Retorna True se a GPU suportar bfloat16, o que permite treinar com menor consumo de memória e mais eficiência.

---
```python
import torch
```


O PyTorch é uma das principais bibliotecas de aprendizado de máquina. No contexto de fine-tuning, o PyTorch é usado para manipular tensores, configurar o dispositivo (CPU/GPU) e gerenciar o treinamento.

Funções comuns em fine-tuning:

- torch.device: Define o dispositivo (CPU ou GPU).
- torch.cuda.is_available: Verifica a disponibilidade da GPU.
- torch.bfloat16: Usado para treinar modelos com precisão bfloat16 se suportado.

---
```python
from datasets import load_dataset
```


A biblioteca datasets da Hugging Face é usada para carregar conjuntos de dados de maneira simples e eficiente. Ela é especialmente útil em tarefas de NLP.

---
```python
from trl import SFTTrainer
```


TRL é uma biblioteca da Hugging Face focada em treinamento com técnicas como Fine-Tuning Supervisionado (SFT) e Reinforcement Learning with Human Feedback (RLHF).

O SFTTrainer (Supervised Fine-Tuning Trainer) facilita o ajuste fino supervisionado de modelos de linguagem.

---
```python
from transformers import TrainingArguments
```


O módulo transformers é fundamental para trabalhar com modelos de linguagem da Hugging Face. O TrainingArguments define os parâmetros de treinamento, como taxa de aprendizado, número de épocas, e estratégias de avaliação.

</details>


In [ ]:
# Se estiver usando o Google Colab, descomente as linhas abaixo
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
BASE_FILE_PATH = '/files'

## Descomprimindo e manipulando o dataset

O arquivo é muito grande, fizemos comprimimos ele para .gz para facilitar a manipulação

In [ ]:
# Decompress a .gz file
input_gz_file = f'${BASE_FILE_PATH}/trn.json.gz'
output_file = f'${BASE_FILE_PATH}/trn.json' # output file path

with gzip.open(input_gz_file, 'rb') as f_in:
    with open(output_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
print(f"File '{input_gz_file}' decompressed successfully to '{output_file}'")


File '/content/drive/My Drive/Colab/tech3/trn.json.gz' decompressed successfully to '/content/drive/My Drive/Colab/tech3/trn.json'


### Gerando o dataset

In [ ]:
file_path = f'${BASE_FILE_PATH}/trn.json'
output_path = f'${BASE_FILE_PATH}/tr_clean.json'

data_list = []
i =0
with open(file_path, 'r') as file:
    for line in file:
        # Parse each line as a JSON object and append to the list
        j = json.loads(line.strip())
        if len(j['title']) > 0 and len(j['content']) > 0:
          data_list.append({key: j[key] for key in ['title', 'content'] if key in j})

df = pd.DataFrame(data_list)

df.to_json(output_path, orient='records')

In [ ]:
## Validate the file
json_file = f'${BASE_FILE_PATH}/tr_clean.json'
df = pd.read_json(json_file)
df.head()

,title,content
0,Girls Ballet Tutu Neon Pink,High quality 3 layer ballet tutu. 12 inches in...
1,Mog's Kittens,Judith Kerr&#8217;s best&#8211;selling adventu...
2,Girls Ballet Tutu Neon Blue,Dance tutu for girls ages 2-8 years. Perfect f...
3,The Prophet,"In a distant, timeless place, a mysterious pro..."
4,Rightly Dividing the Word,--This text refers to thePaperbackedition.


In [6]:
## Asserting torch.cuda is available
print(torch.__version__)
print(torch.cuda.is_available())

2.6.0+cu124
True


In [ ]:
OUTPUT_PATH_DATASET = f"${BASE_FILE_PATH}/formatted_data.json"
COMPLETE_JSON_PATH = f"${BASE_FILE_PATH}/tr_clean.json"

## Iniciando os trabalhos com o modelo

In [ ]:
max_seq_length = 2048
dtype = None
load_in_4bit = True
found_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
]

selected_model = found_models[2] # => "unsloth/llama-3-8b-bnb-4bit"

prompt =  """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

Instruction:
{}

Input:
{}

Response:
{}"""


In [ ]:
def format_dataset_into_model_input(data):

    instructions = []
    inputs = []
    outputs = []

    for index,row in data.iterrows():
        instruction, input_text, response = "What is the content of the product?", row['title'], row['content']
        instructions.append(instruction)
        inputs.append(input_text)
        outputs.append(response)

    formatted_data = {
        "instruction": instructions,
        "input": inputs,
        "output": outputs
    }

    with open(OUTPUT_PATH_DATASET, 'w') as output_file:
        json.dump(formatted_data, output_file, indent=4)
    print(formatted_data)
    print(f"Dataset salvo em {OUTPUT_PATH_DATASET}")

In [ ]:
df = pd.read_json(COMPLETE_JSON_PATH)
df.head()
df.replace('', np.nan, inplace=True)
df = df.dropna(subset=['title', 'content'], how='any')
format_dataset_into_model_input(df)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1390403 [00:00<?, ? examples/s]

In [ ]:
def getModel(model_name):
  model, tokenizer = FastLanguageModel.from_pretrained(
      model_name = model_name,
      max_seq_length = max_seq_length,
      dtype = dtype,
      load_in_4bit = load_in_4bit,
  )
  model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
  )
  return model, tokenizer

<details open>
  <summary>Explicação das os parametros em FastLanguageModel.from_pretrained</summary>


Essa função carrega um modelo de linguagem pré-treinado da Unsloth, otimizado para um carregamento mais rápido e eficiente em memória. Os parâmetros controlam qual modelo é carregado e como ele é configurado.

Parâmetros:

1. model_name: Especifica o nome do modelo pré-treinado a ser carregado. No seu caso, fourbit_models[2] está sendo usado, que corresponde ao modelo "unsloth/llama-3-8b-bnb-4bit". Este é um modelo Llama 3 de 8 bilhões de parâmetros quantizado em 4 bits para reduzir o uso de memória.

2. max_seq_length: Define o comprimento máximo da sequência de entrada que o modelo pode processar. Você está usando a variável max_seq_length, que foi definida como 2048 no seu código. Isso significa que o modelo pode lidar com sequências de até 2048 tokens.

3. dtype: Determina o tipo de dados usado para os pesos do modelo. No seu código, dtype está definido como None. Isso significa que a biblioteca Unsloth escolherá automaticamente o tipo de dados ideal para o seu hardware (provavelmente torch.float16 ou torch.bfloat16).

4. load_in_4bit: Controla se o modelo deve ser carregado usando quantização de 4 bits. Você definiu load_in_4bit como True, o que significa que o modelo será carregado em uma representação de 4 bits, economizando memória.

Retorno:

A função retorna dois objetos:

- model: O modelo de linguagem carregado e configurado.
- tokenizer: O tokenizador associado ao modelo, usado para converter texto em tokens que o modelo pode entender.

</details>

<details open>
  <summary>Explicação das os parametros em FastLanguageModel.get_peft_model</summary>

  A função FastLanguageModel.get_peft_model aplica uma técnica de fine-tuning eficiente, geralmente baseada em LoRA (Low-Rank Adaptation), que permite ajustar um modelo grande modificando apenas um subconjunto de seus parâmetros. Vamos detalhar cada parâmetro:

- model:
É o modelo base que você já carregou e ao qual deseja aplicar o fine-tuning de forma eficiente. Esse modelo será “adaptado” para incorporar os ajustes finos sem alterar todos os seus parâmetros.

- r = 16:
Esse parâmetro define o rank da decomposição de baixo rank aplicada nos módulos alvo. Em termos simples, o valor 16 indica que a adaptação será feita com matrizes de fatoração de rank 16, reduzindo significativamente o número de parâmetros a serem treinados e, consequentemente, os recursos necessários.

- target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]:
Aqui você especifica quais módulos (ou camadas) do modelo serão modificados com o LoRA.

  - "q_proj", "k_proj", "v_proj", "o_proj": Geralmente são os componentes das camadas de atenção dos Transformers, responsáveis por processar as consultas, chaves, valores e projeções de saída.
  - "gate_proj", "up_proj", "down_proj": Podem estar relacionados a adaptações em módulos de adaptação ou transformações adicionais (por exemplo, em modelos que utilizam mecanismos de "gating" ou camadas intermediárias).

- lora_alpha = 16:
Esse é um fator de escala usado para ajustar a magnitude dos updates gerados pela adaptação LoRA. Ele ajuda a balancear a contribuição dos novos parâmetros (de baixo rank) em relação aos pesos originais do modelo, funcionando como um hiperparâmetro de regularização.

- lora_dropout = 0:
Define a taxa de dropout aplicada nas camadas de LoRA. Um valor de 0 significa que o dropout está desativado, ou seja, não há remoção aleatória de unidades durante o treinamento dos parâmetros adaptados. Isso pode ser útil se o overfitting não for uma preocupação ou se outras técnicas de regularização estiverem em uso.

- bias = "none":
Este parâmetro controla como os termos de viés (bias) são tratados durante a adaptação. Com o valor "none", indica-se que os bias do modelo original não serão alterados ou adicionados durante o fine-tuning.

- use_gradient_checkpointing = "unsloth":
Gradient checkpointing é uma técnica para reduzir o uso de memória durante o treinamento, ao salvar intermediários de forma seletiva e recomputá-los quando necessário.
  - O valor "unsloth" sugere que a biblioteca unsloth pode ter uma implementação otimizada ou uma configuração própria para essa funcionalidade.

- random_state = 3407:
Define a semente para a geração de números aleatórios, garantindo a reprodutibilidade dos experimentos. Assim, as inicializações dos novos parâmetros (e quaisquer operações aleatórias) serão consistentes entre execuções.

- use_rslora = False:
Essa flag indica se uma variação do LoRA, chamada RS-LoRA, deve ser utilizada. Com o valor False, a adaptação padrão (LoRA) é aplicada, sem as modificações específicas que o RS-LoRA poderia oferecer.

- loftq_config = None:
Este parâmetro permite passar configurações adicionais relacionadas ao LoftQ, que pode ser uma técnica complementar (possivelmente envolvendo otimizações de quantização ou ajustes específicos) aplicada durante o fine-tuning. O valor None indica que nenhuma configuração extra está sendo aplicada e que os valores padrão serão usados.


> Em resumo, essa função adapta o modelo base para realizar um fine-tuning eficiente, alterando apenas as camadas especificadas (como as projeções em atenção e outras transformações), utilizando um ajuste de baixo rank com controle preciso sobre a escala, regularização (dropout), e estratégias de memória e reprodutibilidade. Cada parâmetro foi pensado para oferecer um balanço entre desempenho, uso de memória e estabilidade do treinamento.


</details>

In [ ]:
model, tokenizer = getModel(selected_model)

==((====))==  Unsloth 2025.3.18: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.3.18 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
def create_prompt_dataset(prompt, tokenizer):
  def formatting_prompts_func(examples):
      instructions = examples["instruction"]
      inputs       = examples["input"]
      outputs      = examples["output"]
      texts = []
      EOS_TOKEN = tokenizer.eos_token
      for instruction, input, output in zip(instructions, inputs, outputs):
          text = prompt.format(instruction, input, output) + EOS_TOKEN
          texts.append(text)
      return { "text" : texts, }
  pass
  dataset = load_dataset("json", data_files=OUTPUT_PATH_DATASET, split = "train")
  dataset = dataset.map(formatting_prompts_func, batched = True,)
  return dataset

In [ ]:
dataset = create_prompt_dataset(prompt, tokenizer)

### Limpando o garbage collection

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

### Treinando o modelo


In [26]:
def getTrainer(model, tokenizer, dataset):
  trainer = SFTTrainer(
      model = model,
      tokenizer = tokenizer,
      train_dataset = dataset,
      dataset_text_field = "text",
      max_seq_length = max_seq_length,
      dataset_num_proc = 2,
      packing = False,
      args = TrainingArguments(
          per_device_train_batch_size = 2,
          gradient_accumulation_steps = 4,
          warmup_steps = 5,
          max_steps = 100,
          learning_rate = 2e-4,
          fp16 = not is_bfloat16_supported(),
          bf16 = is_bfloat16_supported(),
          logging_steps = 1,
          optim = "adamw_8bit",
          weight_decay = 0.01,
          lr_scheduler_type = "linear",
          seed = 3407,
          output_dir = "outputs",
      ),
  )
  return trainer

<details open>
  <summary>Explicação das os parametros em SFTTrainer</summary>

- **model = model**  
  O modelo que será ajustado (fine-tuned). Esse já é um modelo previamente carregado, que passará por adaptações supervisionadas.

- **tokenizer = tokenizer**  
  O tokenizer associado ao modelo, responsável por converter o texto em tokens compreensíveis pelo modelo.

- **train_dataset = dataset**  
  O conjunto de dados de treinamento. Geralmente, esse dataset foi carregado com a biblioteca Hugging Face (por exemplo, via `load_dataset`) e contém os exemplos textuais a serem utilizados.

- **dataset_text_field = "text"**  
  Especifica o nome do campo no dataset que contém o texto que deve ser processado e tokenizado. Aqui, o campo é `"text"`.

- **max_seq_length = max_seq_length**  
  Define o comprimento máximo das sequências que serão geradas após a tokenização. Esse valor ajuda a limitar o tamanho dos inputs passados para o modelo.

- **dataset_num_proc = 2**  
  Número de processos paralelos usados para processar o dataset (por exemplo, durante a tokenização). Usar múltiplos processos pode acelerar o pré-processamento dos dados.

- **packing = False**  
  Indica se os textos devem ser "empacotados" (packing), ou seja, concatenados de maneira a maximizar o uso do comprimento máximo da sequência. Quando `False`, cada exemplo é tratado individualmente.

---

### Parâmetros de Treinamento (TrainingArguments)

Dentro do parâmetro `args`, é passada uma instância de **TrainingArguments** que define diversos hiperparâmetros e configurações de treinamento:

- **per_device_train_batch_size = 2**  
  Define o tamanho do lote (batch size) para cada dispositivo (GPU ou CPU). Aqui, cada dispositivo processará 2 exemplos por vez.

- **gradient_accumulation_steps = 4**  
  Número de passos em que os gradientes serão acumulados antes de atualizar os pesos do modelo. Isso permite simular um tamanho de batch maior sem precisar de tanta memória, pois os gradientes de 4 batches são somados antes da atualização.

- **warmup_steps = 5**  
  Número de passos de aquecimento (warmup) em que a taxa de aprendizado aumenta gradualmente até atingir o valor definido. Essa fase ajuda a estabilizar o treinamento logo no início.

- **max_steps = 60**  
  Define o número máximo de passos de treinamento. Com 60 passos, o treinamento é configurado para ser rápido – comum em exemplos ou testes.

- **learning_rate = 2e-4**  
  Taxa de aprendizado utilizada pelo otimizador. Aqui, o valor é 0.0002.

- **fp16 = not is_bfloat16_supported()**  
  Se `bfloat16` não for suportado pelo hardware, o treinamento utilizará o modo `fp16` (meia precisão), que reduz o consumo de memória e pode acelerar os cálculos.

- **bf16 = is_bfloat16_supported()**  
  Se o hardware suportar `bfloat16`, esse modo é ativado. O `bf16` é uma alternativa ao `fp16` com vantagens em determinadas arquiteturas de GPU.

- **logging_steps = 1**  
  Define a frequência com que os logs são registrados durante o treinamento. Com valor 1, o sistema registra informações a cada passo.

- **optim = "adamw_8bit"**  
  Especifica o otimizador a ser utilizado. Aqui, é usado o **AdamW** em versão 8-bit, que consome menos memória, facilitando o treinamento de modelos grandes.

- **weight_decay = 0.01**  
  Parâmetro de regularização que aplica um decaimento aos pesos, ajudando a prevenir overfitting.

- **lr_scheduler_type = "linear"**  
  Define o tipo de agendador de taxa de aprendizado. Com o scheduler linear, a taxa de aprendizado decresce de forma linear ao longo do treinamento.

- **seed = 3407**  
  Semente para a geração de números aleatórios. Isso garante que os experimentos sejam reprodutíveis, ou seja, que os mesmos resultados possam ser obtidos em execuções diferentes.

- **output_dir = "outputs"**  
  Diretório onde os artefatos do treinamento (como checkpoints, logs e o modelo final) serão salvos.


</details>

In [ ]:
trainer = getTrainer(model, tokenizer, dataset)

In [ ]:
# Trains model
trainer.train()

Map (num_proc=2):   0%|          | 0/1390403 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,390,403 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Step,Training Loss
1,1.922100
2,1.638200
3,1.551000
4,1.822000
5,1.692200
6,1.863100
7,1.714000
8,1.768800
9,1.857600
10,1.493500


## Testando o modelo treinado

In [9]:
def run_prompt(model, tokenizer, prompt, instruction, input):
  FastLanguageModel.for_inference(model)
  prompt_string = prompt.format(
          instruction,
          input,
          "",
      )
  inputs = tokenizer([prompt_string], return_tensors = "pt").to("cuda")
  text_streamer = TextStreamer(tokenizer)
  _ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128, temperature = 0.7, top_p=0.09, use_cache=False)

In [ ]:
run_prompt(model, tokenizer, prompt, "What is the content of the product?", "Girls Ballet Tutu Neon Pink")

You have set `use_cache` to `False`, but cache_implementation is set to dynamic. cache_implementation will have no effect.


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

Instruction:
What is the content of the product?

Input:
Girls Ballet Tutu Neon Pink

Response:
This is a beautiful tutu. It is made of a soft, stretchy, nylon/spandex blend. It is a one piece tutu, with a drawstring waist. It is available in sizes 2T-6X. It is available in the colors shown.<|end_of_text|>


In [ ]:
local_model_path = f"${BASE_FILE_PATH}/traning/${selected_model}"

In [ ]:
## Saving local trained
model.save_pretrained(local_model_path)
tokenizer.save_pretrained(local_model_path)

('/content/drive/My Drive/Colab/tech3/traning/unsloth/llama-3-8b-bnb-4bit/tokenizer_config.json',
 '/content/drive/My Drive/Colab/tech3/traning/unsloth/llama-3-8b-bnb-4bit/special_tokens_map.json',
 '/content/drive/My Drive/Colab/tech3/traning/unsloth/llama-3-8b-bnb-4bit/tokenizer.json')